
<br>
<h2 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> Vaccination Forecasting of India</h2> 
<br>
<br> 
<br>
<div class = 'image'> <img style="float:center; border:5px solid #fed049; width:90%" align=center src = https://ichef.bbci.co.uk/news/976/cpsprodpb/11173/production/_116930007_vaccine_coverage_projection_robinson_map640-nc-2x-nc.png> 
</div>
<br>
<br>
<a href ="https://ichef.bbci.co.uk/news/976/cpsprodpb/11173/production/_116930007_vaccine_coverage_projection_robinson_map640-nc-2x-nc.png" title = "Vaccination Forecasting of India" style = "font-size:20px,color: dimgrey, text-align:left,font-family:serif">Image Source: BBC </a>
<br>

<a id = '0'></a>
<h2 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #007580; color : #fed049; border-radius: 5px 5px; text-align:center; font-weight: bold" >Table of Contents</h2> 

* [1. Initial Glimpse of Data](#1.0)
    * [1.1 Visualization of Geography wise Vaccination ](#1.1)
* [2. Checking daily data of India](#2.0)


<a id = '1.0'></a>
<h2 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 1.0 Initial Glimpse on Data </h2> 

In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import geopandas as gpd

In [ ]:
vaccine_data= pd.read_csv("../input/country-wise-vaccination/country_vaccinations.csv")

In [ ]:
vaccine_data.head()

<a id = '1.1'></a>
<h2 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px">  1.1 Visualization of Geography wise Vaccination </h2> 

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
vaccine_bycountry=vaccine_data.groupby('iso_code',as_index=False).agg({"total_vaccinations":"max"})
vaccine_bycountry.info()

In [ ]:
for_plotting = world.merge(vaccine_bycountry, left_on = 'iso_a3', right_on = 'iso_code')

In [ ]:
ax = for_plotting.dropna().plot(column='total_vaccinations', cmap =    
                                'YlGnBu', figsize=(15,9),   
                                 scheme='quantiles', k=5, legend =  
                                  True);
ax.set_title('Total Vaccine dose', fontdict= 
            {'fontsize':25})
#remove axes
ax.set_axis_off()
#move legend to an empty space
ax.get_legend().set_bbox_to_anchor((.12,.12))


In [ ]:
fullyvaccineper100_bycountry=vaccine_data.groupby('iso_code',as_index=False).agg({"people_fully_vaccinated_per_hundred":"max"})
for_plotting = world.merge(fullyvaccineper100_bycountry, left_on = 'iso_a3', right_on = 'iso_code')
ax = for_plotting.dropna().plot(column='people_fully_vaccinated_per_hundred', cmap =    
                                'YlGnBu', figsize=(15,9),   
                                 scheme='quantiles', k=5, legend =  
                                  True);

ax.set_title('Fully vaccination per hundred', fontdict= 
            {'fontsize':25})
#remove axes
ax.set_axis_off()
#move legend to an empty space
ax.get_legend().set_bbox_to_anchor((.12,.12))

In [ ]:
#daily_vaccinations_per_million
dailydoseper1M_bycountry=vaccine_data.groupby('iso_code',as_index=False).agg({"daily_vaccinations_per_million":"mean"})
for_plotting = world.merge(dailydoseper1M_bycountry, left_on = 'iso_a3', right_on = 'iso_code')
ax = for_plotting.dropna().plot(column='daily_vaccinations_per_million', cmap =    
                                'YlGnBu', figsize=(15,9),   
                                 scheme='quantiles', k=5, legend =  
                                  True);

ax.set_title('Daily dose per million', fontdict= 
            {'fontsize':25})
#remove axes
ax.set_axis_off()
#move legend to an empty space
ax.get_legend().set_bbox_to_anchor((.12,.12))

<a id = '2.0'></a>
<h2 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px">  2.0 Checking daily data of India </h2> 

In [ ]:
vaccine_data_india= vaccine_data[vaccine_data['country']=="India"]

In [ ]:
vaccine_data_india.shape
#https://geographicinsights.iq.harvard.edu/IndiaVaccine

In [ ]:
rel_cols=['date','daily_vaccinations']
dataset=vaccine_data_india.loc[:,rel_cols]
dataset.shape

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 8))
plt.plot(vaccine_data_india['Week'], vaccine_data_india['people_fully_vaccinated'], 'b-', label = 'fully_vaccinated')
#plt.plot(store['Order Date'], store['furniture_sales'], 'b-', label = 'furniture')
plt.plot(vaccine_data_india['Week'], vaccine_data_india['daily_vaccinations'], 'r-', label = 'daily_vaccinations')
plt.xlabel('Week'); plt.ylabel('Vaccines'); plt.title('Fully Vaccinated vs Doses ')
plt.legend();

In [ ]:
dataset.plot()

<a id = '3.0'></a>
<h2 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px">  3.0 Forecasting India's Daily vaccination </h2> 

In [ ]:
dataset.head()

In [ ]:
dataset.columns = ['ds','y']
dataset.head()

In [ ]:
dataset['ds'] = pd.to_datetime(dataset['ds'])

In [ ]:
dataset.dropna(axis=0,inplace=True)

In [ ]:
dataset.columns

In [ ]:
#Initialize Model
model=Prophet()
model.fit(dataset)

In [ ]:
model.component_modes

In [ ]:
### Create future dates of 365 days
future_dates=model.make_future_dataframe(periods=365)

In [ ]:
future_dates.tail()

In [ ]:
prediction=model.predict(future_dates)

In [ ]:
prediction.head()

In [ ]:
### plot the predicted projection
model.plot(prediction)

In [ ]:
#### Visualize Each Components[Trends,yearly]
model.plot_components(prediction)

By Dec 2021, India will be able to admister 1.2B doses. Which will cover around 600-700M people. But Indian population has 65% adult, so roughly 900M people needs to be vaccinated. So by Dec2021, we will be able to vaccinate 70% of the population. The supply is supposed to be raised. 

There are three important things:
1. Supply (SII, BB should produce enough vaccine so that there is no shortage at the vaccination center level)
2. Demand (The CoWin app should work properly to allocate slots to people)
3. Adminstration (Enough centers and doctors should be there to adminsiter the vaccine)
